<a href="https://colab.research.google.com/github/22ita23/LargeLanguageModelTraining/blob/main/Peft_And_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers dataset peft accelerate bitsandbytes #transformers pipeline, dataset to create and load dataset, accelerate to optmize the training, bitsandbytes set bits and bytes of model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer #AutoModelForCausalLM for get model, AutoTon=kenizer for convert word to token, TrainingArguments for Configure the Training, Trainer High level API to train model
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset

In [5]:
data={
    "text": [
        "###Question: What is cricket?\n### Answer: Cricket is a sport",
        "###Question: where did cricket found?\n### Answer: Cricket is found in England",
        "###Question: What are the cricket formats?\n### Answer: Formats of cricket are Test, ODI, T20"
    ]
}
dataset = Dataset.from_dict(data)

In [6]:
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [7]:
def tokenize(batch):
  tokens = tokenizer(
      batch["text"],
      padding = "max_length",
      truncation = True,
      max_length = 128
  )
  tokens["labels"] = tokens["input_ids"].copy()
  return tokens

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit = True,
    device_map = "auto"
)

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [9]:
lora_config = LoraConfig(
    r = 8, # r is adaptive
    lora_alpha = 16, # Scaling input range
    target_modules = ["query_key_value"], #Attention mechecism work on 1 value
    lora_dropout = 0.1, # to stop some node
    bias = "none",# to find error
    task_type = TaskType.CAUSAL_LM #peft tasktype, CAUSAL_LM = Causal Language Model
)

In [10]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [11]:
training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 3,
    per_device_train_batch_size = 1,
    learning_rate = 2e-4,
    logging_steps = 1,
    save_strategy = "no",
    report_to = "none"
)

In [12]:
trainer =Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

trainer.train()

/tmp/ipython-input-12-2570077785.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer =Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,5.840400
2,5.226900
3,4.831600
4,4.447700
5,4.215300
6,3.018600
7,3.547900
8,2.380400
9,3.375200


TrainOutput(global_step=9, training_loss=4.0982429186503095, metrics={'train_runtime': 4.8191, 'train_samples_per_second': 1.868, 'train_steps_per_second': 1.868, 'total_flos': 8364732973056.0, 'train_loss': 4.0982429186503095, 'epoch': 3.0})

In [13]:
model.save_pretrained("lora-finetuned")
tokenizer.save_pretrained("lora-finetuned")

('lora-finetuned/tokenizer_config.json',
 'lora-finetuned/special_tokens_map.json',
 'lora-finetuned/vocab.json',
 'lora-finetuned/merges.txt',
 'lora-finetuned/added_tokens.json',
 'lora-finetuned/tokenizer.json')